In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tbp.monty.frameworks.models.object_model import ObjectModel
from tbp.monty.frameworks.utils.logging_utils import load_stats
from tbp.monty.frameworks.utils.plot_utils import plot_graph

In [ ]:
%matplotlib notebook

In [ ]:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict = pretrain_path + "pretrained_ycb_v4/touch_1lm_10distinctobj/pretrained/"
log_path = os.path.expanduser("~/tbp/results/monty/projects/monty_runs/")

exp_name = "evidence_tests_nomt/"
exp_path = log_path + exp_name

save_path = exp_path + '/stepwise_examples/'
train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=False,
                                                                pretrained_dict=pretrained_dict,
                                                               )

In [ ]:
object_id = 'mug'

In [ ]:
plot_graph(lm_models['pretrained'][0][object_id])
plt.show()

In [ ]:
model = ObjectModel(
            object_id=object_id,
            max_nodes=200,
            max_size=0.1,  # 10cm
            num_cells_per_dim=50,  # -> cell size = 1mm (0.001)
        )

In [ ]:
model.set_graph(lm_models['pretrained'][0][object_id])

In [ ]:
print(model)

In [ ]:
model._location_offset

In [ ]:
obs_count = model._observation_count
print(obs_count.shape)

In [ ]:
plot_graph(model._graph)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
locs = model._location_grid
loc_ids = np.where((locs!=0).all(axis=3))
locs_to_use = locs[loc_ids]
s = ax.scatter(locs_to_use[:,0],
           locs_to_use[:,1],
           locs_to_use[:,2],)
#           c=obs_count[exists[0], exists[1], exists[2]])
ax.set_aspect("equal")
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
exists = np.where(obs_count > 0)
s = ax.scatter(exists[0],
           exists[1],
           exists[2],
          c=obs_count[exists[0], exists[1], exists[2]])

ax.set_aspect("equal")
# ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.show()

In [ ]:
show_feature = "principal_curvatures_log"
feature_id = model.feature_mapping["patch"][show_feature][0]
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
exists = np.where(obs_count > 0)
locs = model._location_grid
loc_ids = np.where((locs!=0).all(axis=3))
locs_to_use = locs[loc_ids]
s = ax.scatter(locs_to_use[:,0],
           locs_to_use[:,1],
           locs_to_use[:,2],
              c=model._feature_grid[exists[0],exists[1],exists[2],feature_id],
              cmap='seismic')
ax.set_aspect("equal")
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.title(show_feature)
plt.show()

In [ ]:
show_feature = "point_normal"
feature_id = model.feature_mapping["patch"][show_feature][:3]
fig = plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
exists = np.where(obs_count > 0)
locs = model._location_grid
loc_ids = np.where((locs!=0).all(axis=3))
locs_to_use = locs[loc_ids]
s = ax.scatter(locs_to_use[:,0],
           locs_to_use[:,1],
           locs_to_use[:,2],)
pn_len = 0.03
for i, pn in enumerate(model._feature_grid[exists[0],exists[1],exists[2],feature_id[0]:feature_id[1]]): 
    plt.plot([locs_to_use[i,0], locs_to_use[i,0] + pn[0] * pn_len],
            [locs_to_use[i,1], locs_to_use[i,1] + pn[1] * pn_len],
            [locs_to_use[i,2], locs_to_use[i,2] + pn[2] * pn_len])
ax.set_aspect("equal")
ax.set_xticks([]), ax.set_yticks([]), ax.set_zticks([])
fig.colorbar(s)
plt.title(show_feature)
plt.show()

In [ ]:
cd_ids = feature_id = model.feature_mapping["patch"]["curvature_directions"]
pn_ids = feature_id = model.feature_mapping["patch"]["point_normal"]
cds = model._graph.x[0,cd_ids[0]:cd_ids[1]]
cd1 = cds[:3]
cd2 = cds[3:]
pn = model._graph.x[0,pn_ids[0]:pn_ids[1]]

In [ ]:
n = 10  # number of vectors
# direction = np.array([1, 2, 3])  # direction in which most vectors point
direction = cd1
opposite_direction = -direction  # direction in which two vectors point

# generate random vectors around the main direction
vectors = np.random.randn(n-2, 3)*0.1 + direction
vectors = vectors / np.linalg.norm(vectors, axis=1)[:, np.newaxis]

# add two vectors in the opposite direction
opposite_vectors = np.random.randn(4, 3) *0.1 + opposite_direction
opposite_vectors = opposite_vectors / np.linalg.norm(opposite_vectors, axis=1)[:, np.newaxis]

# combine all vectors
vectors = np.concatenate([vectors, opposite_vectors], axis=0)

In [ ]:
vectors

In [ ]:
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
for vec in vectors:
    plt.plot([0,vec[0]], [0,vec[1]], [0,vec[2]], c='blue')
ax.set_aspect("equal")
plt.show()

In [ ]:
def unit_vector_mean(u_vecs):
    """Calculate the mean unit vector from a list of them."""
    mean = np.median(u_vecs,axis=0)
    normed_mean = mean / np.linalg.norm(mean)
    return normed_mean

In [ ]:
vectors

In [ ]:
from sklearn.cluster import KMeans

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def unit_vector_mean2(vectors):
    kmeans = KMeans(n_clusters=2).fit(vectors)
    cluster_centers = kmeans.cluster_centers_
    
    # Find the average direction for each cluster
    avg_dirs = [normalize(center) for center in cluster_centers]
    
    # Assign each vector to the cluster with the closest average direction
    assigned_vectors = {0: [], 1: []}
    for v in vectors:
        distances = [np.dot(normalize(v), avg_dir) for avg_dir in avg_dirs]
        cluster = np.argmax(distances)
        assigned_vectors[cluster].append(v)
    
    # Average the vectors within each cluster with their corresponding average direction
    cluster_avgs = []
    for cluster, vecs in assigned_vectors.items():
        aligned_vecs = [normalize(v) if np.dot(v, avg_dirs[cluster]) > 0 else normalize(-v) for v in vecs]
        cluster_avgs.append(np.mean(aligned_vecs, axis=0))
    
    # Take the average of the two cluster averages and normalize the result
    average_vector = normalize(np.mean(cluster_avgs, axis=0))
    return average_vector

In [ ]:
unit_vector_mean2(vectors)

In [ ]:
def get_right_hand_angle(v1, v2, pn):
    # some numpy bug (https://github.com/microsoft/pylance-release/issues/3277)
    # cp = lambda v1, v2: np.cross(v1, v2)
    # a = np.dot(cp(v1, v2), pn)
    a = np.dot(np.cross(v1, v2), pn)
    b = np.dot(v1, v2)
    rha = np.arctan2(a, b)
    return rha
def unit_vector_mean3(vectors, cdir2, pn):
    opposite_dir = get_right_hand_angle(vectors, cdir2, pn) < 0
    vectors[opposite_dir] = -vectors[opposite_dir]
    mean = np.median(vectors,axis=0)
    normed_mean = mean / np.linalg.norm(mean)
    return normed_mean

In [ ]:
unit_vector_mean3(vectors, cd2, pn)

In [ ]:
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
for vec in vectors:
    plt.plot([0,vec[0]], [0,vec[1]], [0,vec[2]], c='blue')
mean_vec = unit_vector_mean(vectors)
plt.plot([0,mean_vec[0]], [0,mean_vec[1]], [0,mean_vec[2]], c='red')
alt_mean_vec = unit_vector_mean2(vectors[:-4])
plt.plot([0,alt_mean_vec[0]], [0,alt_mean_vec[1]], [0,alt_mean_vec[2]], c='green')
ax.set_aspect("equal")
plt.show()

In [ ]:
def pose_vector_mean(pns, cds1, cds2):
    pns_to_use = get_right_hand_angle(pns, cds1[0], cds2[0]) > 0
    if sum(pns_to_use) < len(pns_to_use)//2:
        pns_to_use = np.logical_not(pns_to_use)
    norm_mean = np.mean(pns[pns_to_use],axis=0)
    normed_norm_mean = norm_mean / np.linalg.norm(norm_mean)
    
    cd1_dirs = get_right_hand_angle(cds1, cds2[0], normed_norm_mean) < 0
    cds1[cd1_dirs] = -cds1[cd1_dirs]
    cd1_mean = np.median(cds1,axis=0)
    normed_cd1_mean = cd1_mean / np.linalg.norm(cd1_mean)
    
    cd2_mean = np.cross(normed_norm_mean, normed_cd1_mean)
    normed_cd2_mean = cd2_mean / np.linalg.norm(cd2_mean)
    if get_right_hand_angle(normed_cd1_mean, cd2_mean, normed_norm_mean) < 0:
        normed_cd2_mean = -normed_cd2_mean
    return normed_norm_mean, normed_cd1_mean, normed_cd2_mean

In [ ]:
cds = model._graph.x[5:10,cd_ids[0]:cd_ids[1]]
cds1 = cds[:,:3]
cds2 = cds[:,3:]
pns = model._graph.x[5:10,pn_ids[0]:pn_ids[1]]

In [ ]:
pnmean, cd1mean, cd2mean = pose_vector_mean(pns, cds1, cds2)

In [ ]:
plt.figure()
ax = plt.subplot(1,1,1,projection='3d')
for vec in pns:
    plt.plot([0,vec[0]], [0,vec[1]], [0,vec[2]], c='grey')
plt.plot([0,pnmean[0]], [0,pnmean[1]], [0,pnmean[2]], c='blue')
plt.plot([0,cd1mean[0]], [0,cd1mean[1]], [0,cd1mean[2]], c='red')
plt.plot([0,cd2mean[0]], [0,cd2mean[1]], [0,cd2mean[2]], c='orange')
ax.set_aspect("equal")
plt.show()

## Time Different Matrix Representations

In [ ]:
import numpy as np
from collections import defaultdict
import timeit

def increment_dict(sparse_dict, indices):
    for index_tuple in indices:
        sparse_dict[tuple(index_tuple)] += 1

# Create random indices
num_indices = 500
shape = (50, 50, 50, 50)
indices = np.random.randint(0, 50, size=(num_indices, 4))

# Initialize the defaultdict and numpy array
sparse_dict = defaultdict(int)
dense_array = np.zeros(shape, dtype=int)

# Time the increment_dict function
dict_time = timeit.timeit("increment_dict(sparse_dict, indices)",
                          globals=globals(), number=10)

# Time the np.add.at function
numpy_time = timeit.timeit("np.add.at(dense_array, tuple(indices.T), 1)",
                           globals=globals(), number=10)

print(f"Dictionary increment time: {dict_time:.6f} seconds")
print(f"NumPy array increment time: {numpy_time:.6f} seconds")

In [ ]:
import sys
from pympler import asizeof
# Calculate memory usage
dict_memory = asizeof.asizeof(sparse_dict)
numpy_memory = sys.getsizeof(dense_array)

print(f"Dictionary memory usage: {dict_memory / 1024:.2f} KiB")
print(f"NumPy array memory usage: {numpy_memory / 1024:.2f} KiB")

In [ ]:
%pip install pympler

In [ ]:
import numpy as np
from collections import defaultdict
import timeit

def dict_to_array(sparse_dict, shape):
    dense_array = np.zeros(shape, dtype=int)
    for index_tuple, value in sparse_dict.items():
        dense_array[index_tuple] = value
    return dense_array

def array_to_dict(dense_array):
    sparse_dict = defaultdict(int)
    non_zero_indices = np.transpose(np.nonzero(dense_array))
    for index_tuple in non_zero_indices:
        key = tuple(index_tuple)
        sparse_dict[key] = dense_array[key]
    return sparse_dict

# Create random indices
num_indices = 10000
shape = (50, 50, 50, 50)
indices = np.random.randint(0, 50, size=(num_indices, 4))

# Initialize the defaultdict
sparse_dict = defaultdict(int)
increment_dict(sparse_dict, indices)

def conversion_and_add_at(sparse_dict, indices, shape):
    dense_array = dict_to_array(sparse_dict, shape)
    np.add.at(dense_array, tuple(indices.T), 1)
    new_sparse_dict = array_to_dict(dense_array)
    return new_sparse_dict

# Time the conversion_and_add_at function
conversion_time = timeit.timeit("conversion_and_add_at(sparse_dict, indices, shape)",
                                 globals=globals(), number=100)

print(f"Conversion and np.add.at time: {conversion_time:.6f} seconds")


In [ ]:
import numpy as np
from collections import defaultdict
import timeit

def dict_to_array(sparse_dict, shape):
    dense_array = np.zeros(shape, dtype=int)
    for index_tuple, value in sparse_dict.items():
        dense_array[index_tuple] = value
    return dense_array

def array_to_dict(dense_array):
    sparse_dict = defaultdict(int)
    non_zero_indices = np.transpose(np.nonzero(dense_array))
    for index_tuple in non_zero_indices:
        key = tuple(index_tuple)
        sparse_dict[key] = dense_array[key]
    return sparse_dict

# Create random indices
num_indices = 10000
shape = (50, 50, 50, 50)
indices = np.random.randint(0, 50, size=(num_indices, 4))

# Initialize the defaultdict
sparse_dict = defaultdict(int)
increment_dict(sparse_dict, indices)

def conversion_and_add_at(sparse_dict, indices, shape):
    start_time = timeit.default_timer()
    dense_array = dict_to_array(sparse_dict, shape)
    dict_to_array_time = timeit.default_timer() - start_time

    start_time = timeit.default_timer()
    np.add.at(dense_array, tuple(indices.T), 1)
    add_at_time = timeit.default_timer() - start_time

    start_time = timeit.default_timer()
    new_sparse_dict = array_to_dict(dense_array)
    array_to_dict_time = timeit.default_timer() - start_time

    return dict_to_array_time, add_at_time, array_to_dict_time

# Time the conversion_and_add_at function
num_iterations = 1000
total_dict_to_array_time = 0
total_add_at_time = 0
total_array_to_dict_time = 0

for _ in range(num_iterations):
    dict_to_array_time, add_at_time, array_to_dict_time = conversion_and_add_at(sparse_dict, indices, shape)
    total_dict_to_array_time += dict_to_array_time
    total_add_at_time += add_at_time
    total_array_to_dict_time += array_to_dict_time

print(f"Total dict_to_array time: {total_dict_to_array_time:.6f} seconds")
print(f"Total np.add.at time: {total_add_at_time:.6f} seconds")
print(f"Total array_to_dict time: {total_array_to_dict_time:.6f} seconds")


In [ ]:
import torch
import numpy as np
from collections import defaultdict
import timeit

def increment_dict(sparse_dict, indices):
    for index_tuple in indices:
        sparse_dict[tuple(index_tuple)] += 1

def increment_torch_sparse(sparse_tensor, indices):
    unique_indices, counts = np.unique(indices, axis=0, return_counts=True)
#     for i, idx in enumerate(unique_indices):
#         print(f"{idx}: {counts[i]}")
    new_indices = torch.tensor(unique_indices.T, dtype=torch.long)
    new_values = torch.tensor(counts, dtype=torch.int64)
    new_sparse_tensor = torch.sparse_coo_tensor(new_indices, new_values, sparse_tensor.shape)
    return sparse_tensor + new_sparse_tensor



# Create random indices
num_indices = 500
shape = (50, 50, 50, 50)
indices = np.random.randint(0, 50, size=(num_indices, 4))

# Initialize the defaultdict, numpy array, and torch sparse tensor
sparse_dict = defaultdict(int)
dense_array = np.zeros(shape, dtype=int)
sparse_tensor = torch.sparse_coo_tensor(torch.zeros((4, 0), dtype=torch.long), torch.tensor([]), size=shape)

# Time the increment_dict function
dict_time = timeit.timeit("increment_dict(sparse_dict, indices)",
                          globals=globals(), number=1000)

# Time the np.add.at function
numpy_time = timeit.timeit("np.add.at(dense_array, tuple(indices.T), 1)",
                           globals=globals(), number=1000)

# Time the increment_torch_sparse function
torch_sparse_time = timeit.timeit("increment_torch_sparse(sparse_tensor, indices)",
                                  globals=globals(), number=1000)

print(f"Dictionary increment time: {dict_time:.6f} seconds")
print(f"NumPy array increment time: {numpy_time:.6f} seconds")
print(f"Torch sparse tensor increment time: {torch_sparse_time:.6f} seconds")


In [ ]:
import sys
from pympler import asizeof
# Calculate memory usage
dict_memory = asizeof.asizeof(sparse_dict)
numpy_memory = sys.getsizeof(dense_array)
torch_memory = sys.getsizeof(sparse_tensor)

print(f"Dictionary memory usage: {dict_memory / 1024:.2f} KiB")
print(f"NumPy array memory usage: {numpy_memory / 1024:.2f} KiB")
print(f"Torch tensor memory usage: {torch_memory / 1024:.2f} KiB")